# Rotor condition classification

## Import Necessary Libraries

In [4]:
## Import necessary libraries
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
from scipy.fftpack import fft
from sklearn.decomposition import PCA

## Import libraries for the model
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## Set path for saving model training results  
import os
os.makedirs('./result', exist_ok=True)

## Set Cuda for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Set random seed
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# Set seed
seed_val = 77
set_seed(seed_val)

cuda


## Selecting Data Columns
* Tag names are loaded in sequential order.
* The process of selecting the required tag names from the tag name list.

In [5]:
# Function to display tag names
def show_column(URL):
    
    # Load tag name data
    df = pd.read_csv(URL)
    
    # Convert to list format
    df = df.values.reshape(-1)
    
    return df.tolist()

In [6]:
## Set parameters for displaying tag names
table = 'rotor'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get-tag-names.tql?table={table}'

## Generate tag name list 
name = show_column(NAME_URL)

In [7]:
name

['g1_sensor1_normal',
 'g1_sensor1_type1',
 'g1_sensor1_type2',
 'g1_sensor1_type3',
 'g1_sensor2_normal',
 'g1_sensor2_type1',
 'g1_sensor2_type2',
 'g1_sensor2_type3',
 'g1_sensor3_normal',
 'g1_sensor3_type1',
 'g1_sensor3_type2',
 'g1_sensor3_type3',
 'g1_sensor4_normal',
 'g1_sensor4_type1',
 'g1_sensor4_type2',
 'g1_sensor4_type3',
 'g2_sensor1_normal',
 'g2_sensor1_type1',
 'g2_sensor1_type2',
 'g2_sensor1_type3',
 'g2_sensor2_normal',
 'g2_sensor2_type1',
 'g2_sensor2_type2',
 'g2_sensor2_type3',
 'g2_sensor3_normal',
 'g2_sensor3_type1',
 'g2_sensor3_type2',
 'g2_sensor3_type3',
 'g2_sensor4_normal',
 'g2_sensor4_type1',
 'g2_sensor4_type2',
 'g2_sensor4_type3']

## Converting TAG Name Format
* After checking all the Tag Names from the rotor dataset in the previous step, extract only the columns to be used and convert them into parameter format.
* Use tag names related to the g1

In [8]:
# Set the desired tag names
tags = name[:16]

# Wrap each item in the list with single quotes and separate with commas
tags_ = ",".join(f"'{tag}'" for tag in tags)

# Check the selected tag names
print(tags_)

'g1_sensor1_normal','g1_sensor1_type1','g1_sensor1_type2','g1_sensor1_type3','g1_sensor2_normal','g1_sensor2_type1','g1_sensor2_type2','g1_sensor2_type3','g1_sensor3_normal','g1_sensor3_type1','g1_sensor3_type2','g1_sensor3_type3','g1_sensor4_normal','g1_sensor4_type1','g1_sensor4_type2','g1_sensor4_type3'


## Load Rotor Dataset
* Load the entire dataset upon data loading.

    * Label description:

        * normal: Normal
        * type1: Rotational imbalance on Disk 2 (bolt and nut attached at the 270-degree position)
        * type2: Support imbalance on Support 4
        * type3: Combination of Type 1 and Type 2

In [9]:
# Data loading parameter settings

# Set the tag table name
table = 'rotor'
# Set the tag names
name = quote(tags_, safe=":/")
# Set the time format  
timeformat = quote('2006-01-02 15:04:05.000000')
# Set the data start time
start_time = quote('2024-01-01 00:00:00')
# Set the data end time
end_time = quote('2024-01-01 00:02:19.999')

In [10]:
# Data loading function
def data_load(table, name, start_time, end_time, timeformat):
    
    # Load data  
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # Convert to data grouped by the time
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # Set TIME column
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # Create an empty DataFrame
    df_result = pd.DataFrame()
    
    # Interpolate data for each tag name
    for i in range(len(df.columns[1:])):
        
        # Set time
        start = pd.to_datetime(unquote(start_time))
        end = pd.to_datetime(unquote(end_time))
        
        df_ = df.iloc[ : , [0] + list(range(i+1, i+2))].dropna()
        
        # Create a new time range for interpolation (1000 points for each second)
        # In this case, the original data was measured at 1 ms intervals, so we create 1000 points at 1-second intervals
        # Generate range from 0 to 140 -> 140,000
        new_time_range = pd.date_range(start=start, end=end, freq='1ms')
        new_time_range_ = pd.date_range(start=start, end=end, freq='1s')

        # Use linear interpolation to fill in the data
        # Convert datetime to numeric (epoch time in seconds)
        time_numeric = pd.to_numeric(df_['TIME'])
        new_time_numeric = pd.to_numeric(new_time_range)

        value = df_[df_.columns[1:].values]

        # Create linear interpolation object
        interpolator = interp1d(time_numeric, value.values.reshape(-1), kind='linear', fill_value='extrapolate')
        interpolated_values = interpolator(new_time_numeric)
        interpolated_values = np.clip(interpolated_values, min(value.values), max(value.values))

        # Create DataFrame
        df_remake = pd.DataFrame(interpolated_values.reshape(-1,1000))
        df_remake['time'] = new_time_range_
        df_remake['sensor'] = f'{df_.columns[1:].item()}'

        # Specify columns to move and the new order
        cols = df_remake.columns.tolist()
        cols.insert(0, cols.pop(cols.index('time')))
        cols.insert(1, cols.pop(cols.index('sensor')))
        df_remake = df_remake[cols]

        # Append to the empty DataFrame
        df_result = pd.concat([df_result, df_remake], ignore_index=True)
        
    # Sort by time
    df_result = df_result.sort_values(by='time').reset_index(drop=True)
        
    return df_result

In [11]:
# Load data
df = data_load(table, name, start_time, end_time, timeformat)
df

,time,sensor,0,1,2,3,4,5,6,7,...,990,991,992,993,994,995,996,997,998,999
0,2024-01-01 00:00:00,g1_sensor1_normal,-0.853307,-0.524641,-0.003741,-0.297684,-0.091203,-0.045372,-0.060902,0.508235,...,0.139246,0.696438,-0.508470,0.264728,-0.399669,-0.316289,-0.763595,-1.010909,-0.718536,-0.720669
1,2024-01-01 00:00:00,g1_sensor1_type2,0.555219,-0.153753,-0.197844,-0.972652,-0.913384,-1.390481,-1.414697,-1.586338,...,1.346492,1.246483,1.169959,1.310134,1.402345,1.329445,1.220657,1.077944,0.662889,0.673957
2,2024-01-01 00:00:00,g1_sensor1_type3,3.919664,3.713706,2.698885,1.338952,0.701167,-0.333580,-0.488003,-2.033326,...,6.453300,6.381317,6.292205,6.023239,5.512099,5.003446,3.997718,3.273269,1.813393,1.391992
3,2024-01-01 00:00:00,g1_sensor2_normal,0.048823,-0.029477,-0.004731,0.009673,0.096184,0.009673,0.009673,0.024096,...,1.224365,0.973323,0.398562,0.589916,0.520218,-0.442281,-0.729798,-1.159330,-1.270378,-1.268479
4,2024-01-01 00:00:00,g1_sensor2_type1,-1.054255,-1.173785,-1.142896,-1.357203,-0.848193,-0.319767,-1.745371,0.480831,...,0.252693,-1.117656,0.199578,-0.209522,-0.181900,-0.995153,-0.854758,0.305171,-1.650853,-0.284521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2024-01-01 00:02:19,g1_sensor1_type2,0.101884,0.586425,0.301300,0.480463,0.595780,0.716434,0.976936,0.963343,...,-0.073815,0.539034,0.110189,-0.139597,0.531263,0.664176,1.050553,0.868198,0.792942,1.197780
2236,2024-01-01 00:02:19,g1_sensor1_type1,1.904225,2.580567,2.227736,2.198483,2.072673,2.332377,2.170863,1.553462,...,0.112187,-0.015991,-0.595242,-0.885469,-0.792080,-1.389177,-1.368793,-2.281878,-1.887351,-2.443287
2237,2024-01-01 00:02:19,g1_sensor1_normal,0.917832,0.091463,0.723296,0.487312,1.017593,0.800933,0.449253,0.847483,...,0.261695,-0.025523,-0.376261,-0.542628,-0.685341,-1.057457,-0.513635,-1.085233,-0.677111,-1.447941
2238,2024-01-01 00:02:19,g1_sensor4_type2,-0.734751,-1.679474,-0.205754,-0.770393,0.273960,0.291936,0.355024,0.864662,...,2.318283,1.554217,1.711223,0.320252,0.483345,-0.423499,-1.127343,-0.805819,-1.208654,-1.151504


In [12]:
# Label mapping

# Label mapping dictionary
label_mapping = {
    'normal': 0,
    'type1': 1,
    'type2': 2,
    'type3': 3
}

# Function to extract labels from column names
def get_label(column_name):
    column_name = str(column_name)
    for key in label_mapping.keys():
        if key in column_name:
            return label_mapping[key]
    return None 

In [13]:
# Apply labels to each column and create a new series
labels = pd.Series(df['sensor']).map(get_label)

# Add the label to the DataFrame
df['label'] = labels.values

# Print the count of each label
print(df['label'].value_counts())

label
0    560
2    560
3    560
1    560
Name: count, dtype: int64


In [14]:
# Split the data into train, validation, and test sets

# Extract data for each sensor
# 100 samples for training
# 20 samples for validation
# 20 samples for testing
train = pd.DataFrame()
valid = pd.DataFrame()
test = pd.DataFrame()

for i in range(len(df['sensor'].value_counts().index)):
    
    df_train = df[df['sensor'] == df['sensor'].value_counts().index[i]][:100].iloc[:,2:]
    df_valid = df[df['sensor'] == df['sensor'].value_counts().index[i]][100:120].iloc[:,2:]
    df_test = df[df['sensor'] == df['sensor'].value_counts().index[i]][120:].iloc[:,2:]
    
    train = pd.concat([train, df_train])
    valid = pd.concat([valid, df_valid])
    test = pd.concat([test, df_test])
    
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

print(train['label'].value_counts())
print(valid['label'].value_counts())
print(test['label'].value_counts())

label
0    400
2    400
3    400
1    400
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64


## Data Preprocessing

* 1 hanning window
* 2 FFT
* 3 MinMax Scaling

### 1. Applying Hanning Window

In [15]:
# Hanning window function setup 
def set_hanning_window(sample_rate, df):
    
    # Generate Hanning window
    hanning_window = np.hanning(sample_rate)

    # Apply Hanning window to each row
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

In [16]:
# Sampling period -> Number of data points per second
window_length = len(df.columns[2:-1])

# Applying Hanning Window
train_ = set_hanning_window(window_length, train.iloc[:,:-1])
valid_ = set_hanning_window(window_length, valid.iloc[:,:-1])
test_ = set_hanning_window(window_length, test.iloc[:,:-1])

### 2. Applying FFT (Fast Fourier Transform)

In [17]:
# FFT transformation function
def change_fft(sample_rate, df):
    # Total number of samples in the signal
    N = sample_rate
    
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    # Apply FFT to each row
    for i in range(df.shape[0]):
        
        # Calculate FFT for each row
        yf = fft(df.iloc[i].values)
        
        # Compute the absolute value of the FFT results and normalize (only the meaningful part)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # Convert FFT results to a DataFrame
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

In [18]:
# Sampling period -> Number of data points per second
sampling_rate = len(df.columns[2:-1])

# Applying FFT (Fast Fourier Transform)
train_ = change_fft(sampling_rate, train_)
valid_ = change_fft(sampling_rate, valid_)
test_ = change_fft(sampling_rate, test_)

### 3. Applying MinMaxScaler

In [19]:
# Scaler Setup
scaler = MinMaxScaler()

# Apply Scaler
train_ = scaler.fit_transform(train_.values)
valid_ = scaler.transform(valid_.values)
test_ = scaler.transform(test_.values)

# Set DataFrames
train_scaled = pd.DataFrame(train_)
valid_scaled = pd.DataFrame(valid_)
test_scaled = pd.DataFrame(test_)

# Add label
train_scaled['label'] = train['label'].values
valid_scaled['label'] = valid['label'].values
test_scaled['label'] = test['label'].values

print(train_scaled['label'].value_counts())
print(valid_scaled['label'].value_counts())
print(test_scaled['label'].value_counts())

label
0    400
2    400
3    400
1    400
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64


## Dataset & Loader Setup

In [20]:
class Rotor_Dataset(Dataset):

    def __init__(self, df):
        self.freq_data = df.iloc[:,:-1]
        self.label = df.iloc[:,-1:].squeeze()

    def __len__(self):
        return len(self.freq_data)

    def __getitem__(self, index):

        input_time_data = self.freq_data.iloc[index,:]
        input_time_data = torch.Tensor(input_time_data).expand(1, input_time_data.shape[0])
        label = self.label[index]

        return input_time_data, label

In [21]:
# Set up datasets  
train_ = Rotor_Dataset(train_scaled)
valid_ = Rotor_Dataset(valid_scaled)
test_ = Rotor_Dataset(test_scaled)

# Set up data loaders
train_dataloader = DataLoader(train_, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_, batch_size=1, shuffle=True)

In [22]:
# Verify DataLoader application and check the shape of the input data
print(list(train_dataloader)[0][0].shape)

torch.Size([8, 1, 501])


## Model Configuration
* Using ResNet1d model.

In [23]:
## ResNet 1D Model Setup
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [24]:
# Model configuration parameters
# Learning rate
learning_rate = 0.01

# Model configuration
model = ResNet1D(ResidualBlock, [2, 2, 2, 2], num_classes=4).to(device)

# Configure loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check the model architecture
print(model)

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## Model Training

* Save the model with the Best F1 Score based on the validation data during training.

In [25]:
# Initialize training loss
train_loss = []
# Initialize training accuracy
train_acc = []
# Initialize total step
total_step = len(train_dataloader)
# Set number of epochs
epoch_in = trange(100, desc='training')
# Initialize best F1 Score value
best_f1= 0

# Start model training
for epoch in epoch_in:
    model.train()
    running_loss = 0.0
    correct = 0
    total=0

    preds_ = []
    targets_ = []

    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data[0].to(device).float()
        labels = train_data[1].to(device).long().squeeze()

        optimizer.zero_grad()

        # Input to the model
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Set label predictions 
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)
        
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')
    
    # Perform validation at the end of each epoch and save the model with the best performance
    with torch.no_grad():
        model.eval()
        
        for batch_idx, valid_data in enumerate(valid_dataloader):

            inputs_v = valid_data[0].to(device).float()
            labels_v = valid_data[1].to(device).long().squeeze() 
            
            outputs_v = model(inputs_v)
            
            # Set label predictions
            _,pred_v = torch.max(outputs_v, dim=1)
            target_v = labels_v.view_as(pred_v)
            
            preds_.append(pred_v)
            targets_.append(target_v)
            
        # Combine predictions and labels collected from all batches
        preds_ = torch.cat(preds_).detach().cpu().numpy()
        targets_ = torch.cat(targets_).detach().cpu().numpy()
        
        f1score = f1_score(targets_, preds_,  average='macro')
        if best_f1 < f1score:
            best_f1 = f1score
            # Save the best model 
            with open("./result/Rotor_1d_ResNet_General.txt", "a") as text_file:
                print('epoch=====',epoch, file=text_file)
                print(classification_report(targets_, preds_, digits=4), file=text_file)
            torch.save(model, f'./result/Rotor_1d_ResNet_General.pt') 
        epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")

training:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 1.3326324446499347, train acc: 45.3125

train loss: 1.0101663764566182, train acc: 70.8750

train loss: 0.8098243155765036, train acc: 83.9375

train loss: 0.6831355629535392, train acc: 88.5000

train loss: 0.5867879580068401, train acc: 92.5000

train loss: 0.5234523606393485, train acc: 92.5000

train loss: 0.47297797634657135, train acc: 94.5625

train loss: 0.4280194197935634, train acc: 96.8125

train loss: 0.3937211614839422, train acc: 95.7500

train loss: 0.36322635618166527, train acc: 96.3125

train loss: 0.3389954990106211, train acc: 96.6875

train loss: 0.3187843384031899, train acc: 96.3750

train loss: 0.29943007610246863, train acc: 97.6875

train loss: 0.28015993621888413, train acc: 99.1875

train loss: 0.2678981447784741, train acc: 96.5625

train loss: 0.2576470710222463, train acc: 96.6875

train loss: 0.2463012431532875, train acc: 97.9375

train loss: 0.2344595020104508, train acc: 99.2500

train loss: 0.22256130556602183, train acc: 99.8125

train 

## Model Testing

In [26]:
# Load the best model
model_ = torch.load(f'./result/Rotor_1d_ResNet_General.pt') 

In [27]:
# Model testing
preds_test = []
target_test = []
with torch.no_grad():
    model_.eval()
    for batch_idx, test_data in enumerate(test_dataloader):
        inputs_t = test_data[0].to(device).float()
        labels_t =  test_data[1].to(device).long().squeeze() 
        
        outputs_t = model_(inputs_t)
        
        _,pred_t = torch.max(outputs_t, dim=1)
        targets_t = labels_t.view_as(pred_t).to(device)

        preds_test.append(pred_t)
        target_test.append(targets_t)
        
    # Combine predictions and labels collected from all batches
    preds_test = torch.cat(preds_test).detach().cpu().numpy()
    target_test = torch.cat(target_test).detach().cpu().numpy()

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


## Model Performance Evaluation

In [28]:
print(classification_report(target_test, preds_test))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94        80
           1       0.96      0.93      0.94        80
           2       1.00      1.00      1.00        80
           3       1.00      1.00      1.00        80

    accuracy                           0.97       320
   macro avg       0.97      0.97      0.97       320
weighted avg       0.97      0.97      0.97       320

